In [9]:
import numpy as np
import pandas as pd

def entropy(y):
    """calculating tag y's entropy"""
    value_counts = np.bincount(y)
    probabilities = value_counts[value_counts > 0] / len(y)
    return -np.sum(probabilities * np.log2(probabilities))


def information_gain(X_column, y):
    """Caluculate info gain based on certain division"""
    total_entropy = entropy(y)
    values = np.unique(X_column)
    
    weighted_entropy = 0
    for val in values:
        subset_y = y[X_column == val]
        weight = len(subset_y) / len(y)
        weighted_entropy += weight * entropy(subset_y)

    return total_entropy - weighted_entropy


In [10]:
from sklearn.datasets import load_iris
import pandas as pd

# load the data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# calculate the info gain for each feature
print("Information Gain for each feature:")
best_feature = None
best_gain = -1

for i, column in enumerate(X.columns):
    gain = information_gain(X[column], y)
    print(f"{i}. {column} --> Info Gain = {gain:.4f}")
    if gain > best_gain:
        best_gain = gain
        best_feature = column

print(f"\nBest feature to split: {best_feature} (Gain = {best_gain:.4f})")

Information Gain for each feature:
0. sepal length (cm) --> Info Gain = 0.8769
1. sepal width (cm) --> Info Gain = 0.5166
2. petal length (cm) --> Info Gain = 1.4463
3. petal width (cm) --> Info Gain = 1.4359

Best feature to split: petal length (cm) (Gain = 1.4463)


In [11]:
class Node:
    def __init__(self, feature=None, threshold=None, children=None, value=None):
        """
        dicision tree class
        
        """
        self.feature = feature
        self.threshold = threshold
        self.children = children or {}
        self.value = value

    def is_leaf(self):
        return self.value is not None

In [12]:
# tree sturcture printer

def print_tree(node, depth=0):
    indent = "  " * depth  

    if node.is_leaf():
        print(f"{indent}Predict → Class {node.value}")
        return

    for val, child in node.children.items():
        print(f"{indent}If {node.feature} == {val}:")
        print_tree(child, depth + 1)

In [13]:
from graphviz import Digraph

def visualize_tree(node, dot=None, node_id=0):
    if dot is None:
        dot = Digraph()

    current_id = str(node_id)

    if node.is_leaf():
        label = f"Predict: {node.value}"
    else:
        label = f"{node.feature}"

    dot.node(current_id, label)

    for i, (val, child) in enumerate(node.children.items()):
        child_id = f"{node_id}_{i}"  
        dot.edge(current_id, child_id, label=str(val))
        visualize_tree(child, dot, child_id)

    return dot


In [14]:
# define the tree building function

def build_tree(X, y, depth=0):
   
    if len(set(y)) == 1:
        return Node(value=y[0])
    
    if X.shape[1] == 0:
        most_common = np.bincount(y).argmax()
        return Node(value=most_common)

    best_gain = -1
    best_feature = None

    for column in X.columns:
        gain = information_gain(X[column], y)
        if gain > best_gain:
            best_gain = gain
            best_feature = column

    if best_gain == 0:
        most_common = np.bincount(y).argmax()
        return Node(value=most_common)

    root = Node(feature=best_feature)
    for val in np.unique(X[best_feature]):
        idx = X[best_feature] == val
        child = build_tree(X[idx].drop(columns=[best_feature]), y[idx], depth + 1)
        root.children[val] = child

    return root


tree = build_tree(X, y)
print_tree(tree)

If petal length (cm) == 1.0:
  Predict → Class 0
If petal length (cm) == 1.1:
  Predict → Class 0
If petal length (cm) == 1.2:
  Predict → Class 0
If petal length (cm) == 1.3:
  Predict → Class 0
If petal length (cm) == 1.4:
  Predict → Class 0
If petal length (cm) == 1.5:
  Predict → Class 0
If petal length (cm) == 1.6:
  Predict → Class 0
If petal length (cm) == 1.7:
  Predict → Class 0
If petal length (cm) == 1.9:
  Predict → Class 0
If petal length (cm) == 3.0:
  Predict → Class 1
If petal length (cm) == 3.3:
  Predict → Class 1
If petal length (cm) == 3.5:
  Predict → Class 1
If petal length (cm) == 3.6:
  Predict → Class 1
If petal length (cm) == 3.7:
  Predict → Class 1
If petal length (cm) == 3.8:
  Predict → Class 1
If petal length (cm) == 3.9:
  Predict → Class 1
If petal length (cm) == 4.0:
  Predict → Class 1
If petal length (cm) == 4.1:
  Predict → Class 1
If petal length (cm) == 4.2:
  Predict → Class 1
If petal length (cm) == 4.3:
  Predict → Class 1
If petal length (cm)

In [15]:
def predict_sample(node, sample):
    
    if node.is_leaf():
        return node.value

    feature_value = sample[node.feature]
    child = node.children.get(feature_value)

    if child is None:

        return None

    return predict_sample(child, sample)

def predict_all(node, X):

    return [predict_sample(node, row) for _, row in X.iterrows()]

def accuracy(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    valid = y_pred != None
    return (y_true[valid] == y_pred[valid]).sum() / valid.sum()

tree = build_tree(X, y)

y_pred = predict_all(tree, X)

acc = accuracy(y, y_pred)
print(f"Accuracy on training data: {acc * 100:.2f}%")

Accuracy on training data: 100.00%


In [16]:
dot = visualize_tree(tree)
dot.render("scratch_tree", format="png", cleanup=True)

print("scratch_tree.png")

scratch_tree.png
